In [102]:
#Importing Libraries.
import time
import requests
import pandas as pd
from tqdm import tqdm 

from bs4 import BeautifulSoup
from selenium import webdriver

# Selenium
browser = webdriver.Chrome()
browser.get('https://stockmages.netlify.app')

#Now the main point of concern is what should be the stop and for the for loop. The start should be 0.
#Finding the stop is a manual process. We should use a random number for stops and see how much scrolling takes place. If it doesn’t scroll to the bottom, we have to increase the value of the stop. 
#Infinite Scroll to reach the bottom 
for i in tqdm(range(0,500000,1000)):
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)
#Using this much range will lead ypu to the bottom of the path

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:54<00:00,  9.18it/s]


In [123]:
# Scraping the Page
soup = BeautifulSoup(browser.page_source,'html.parser')

#Scraping the Image Details
data = []
for sp in tqdm(soup.find_all('div', class_ = 'container')):
    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9]) 
    data.append([img_link, tags, likes, comments])

#Creating a dataframe and saving it
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])
df.to_csv('Images.csv', index = False)

100%|███████████████████████████████████████████████████████████████████████████| 1044/1044 [00:00<00:00, 12039.21it/s]


In [3]:
from tqdm import tqdm
import pandas as pd
import os
os.mkdir('Images')
df=pd.read_csv('Images.csv')
import requests
def download(link,img_path):
    res=requests.get(link)
    fd=open(img_path,'wb')
    fd.write(res.content)
    fd.close()

paths=[]
img_ids=[]
ref=100000
i=1

for link in tqdm(df['img_link']):
    img_id='A'+str(i+ref)[1:]
    i+=1
    img_path='Images/'+img_id+'.jpg'
    download(link,img_path)
    paths.append(img_path)
    img_ids.append(img_id)

df['img_id']=img_ids
df['img_path']=paths
df.to_csv('Images_Description.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1044/1044 [05:17<00:00,  3.28it/s]


In [5]:
import os
import pandas as pd
df=pd.read_csv('Images_Description.csv')
df.head()

,img_link,tags,likes,comments,img_id,img_path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,A00001,Images/A00001.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,A00002,Images/A00002.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,A00003,Images/A00003.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,A00004,Images/A00004.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,A00005,Images/A00005.jpg


### 1. Remove Unnecessary Columns

In [7]:
del df['img_link']
del df['img_id']
del df['likes']
del df['comments']

df.head()

,tags,img_path
0,"Clouds, Sky, Atmosphere, Blue Sky",Images/A00001.jpg
1,"Bird, Ornithology, Hummingbird",Images/A00002.jpg
2,"Sea, Rainbow, Rainfall, Subtropical",Images/A00003.jpg
3,"Cherry Blossoms, Road, Japan, Sakura",Images/A00004.jpg
4,"Cape Marguerite, Flower, Plant",Images/A00005.jpg


### 2. Finding all the tags

In [9]:
t=[]
for tags in (df['tags']):
    try:
     t+= ([tag.strip() for tag in tags.split(',')])
    except:
        pass

for ts in t:
    if ts=='' or ts=='nan':
        t.remove(ts)

tags=list(set(t))

### 3.Creating Folders for each Tag

In [279]:
from tqdm import tqdm

In [13]:
os.mkdir('Dataset')
for tag in  tqdm(tags):
    try:
      os.mkdir('Dataset/'+tag)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1657/1657 [00:00<00:00, 3602.20it/s]


### 4.Copying Images to Their Tag Folder

In [ ]:
import os
import hashlib
from shutil import copyfile as copy
from tqdm import tqdm

done=0
error = 0
for data in tqdm(df.values):
    src = str('/content/Images/'+data[1].split('/')[1].strip())
    if isinstance(data[0], str):
        for tag in data[0].split(','):

            dst = ('/content/Dataset/' +tag.strip()+ '/' + data[1].split('/')[1]).strip()

            try:
              copy(src,dst)
              done+=1

            except:
              pass
              error+=1

print("Error :",error)
print("Done :",done)

### 5.Checking No Of Folders

In [ ]:
folders=os.listdir('Dataset')
print(len(folders))

### 6. Checking No of Images in Each Folder

In [ ]:
folder_=[]
freq=[]

for folder in tqdm(folders):
  try:
    freq.append(len(os.listdir('Dataset/'+folder)))
    folder_.append(folder)

  except:
    pass

In [ ]:
print(len(folder_))
print(len(freq))

### 7.Top 10 Folders with most number of Images

In [ ]:
df_=pd.DataFrame()
df_['Folder']=folder_
df_['Frequency']=freq
df_.sort_values(by='Frequency',ascending=False).head(10)

### 8. Removing the Folders with less than 5 Images

In [ ]:
(df_['Frequency']<5).sum()

In [ ]:
import shutil

for folder in tqdm(df_['Folder'][df_['Frequency']<5]):
  try:
    shutil.rmtree('Dataset/'+folder)
  except:
    pass

In [ ]:
folders=os.listdir('Dataset')
print(len(folders))